In [3]:
from linearmodels import PanelOLS
import pandas as pd
import numpy as np
import numpy.linalg as la
import quantecon as qe
import statsmodels as sm
from rpy2.robjects import r, pandas2ri


In [4]:
%store -r digital

In [5]:
digital = digital.copy()
digital.describe()

,date,Access Electricity,Cellular %,Fixed broadband %,GDP pcp PPP,Landline %,Secure Servers,Urbanisation,internet_users
count,4136.000000,3685.000000,4049.000000,2599.000000,4010.000000,4049.000000,2546.000000,4136.000000,3895.000000
mean,2005.500000,76.185665,55.453445,8.101887,14513.301732,19.004614,5031.332600,55.308416,23.632575
std,6.345056,32.351223,52.530423,10.990397,17766.001536,18.943349,31519.654906,24.174969,27.109598
min,1995.000000,0.014814,0.000000,0.000000,272.271966,0.000000,1.000000,7.211000,0.000000
25%,2000.000000,54.499317,4.521988,0.165319,2655.234687,2.722144,11.000000,34.648750,1.666235
50%,2005.500000,96.366173,42.874766,2.388865,7758.475175,12.710057,72.500000,55.259500,10.600000
75%,2011.000000,100.000000,98.895536,12.454840,20153.544633,30.276428,620.250000,74.376750,40.425000
max,2016.000000,100.000000,332.090701,61.738849,140037.115597,110.191151,530309.000000,100.000000,98.323610


In [8]:
digital.head()

,country,date,Access Electricity,Cellular %,Fixed broadband %,GDP pcp PPP,Landline %,Secure Servers,Urbanisation,internet_users,Code,Region,cgroup
0,Afghanistan,2016,NaN,66.003744,0.026890,1876.544682,0.348892,49.0,27.132,10.595726,AFG,South Asia,L
1,Afghanistan,2015,NaN,61.577682,0.022080,1861.124332,0.343677,46.0,26.703,8.260000,AFG,South Asia,L
2,Afghanistan,2014,89.500000,58.845471,0.004795,1875.447407,0.325861,32.0,26.282,7.000000,AFG,South Asia,L
3,Afghanistan,2013,75.154373,55.012226,0.004910,1877.411953,0.313466,30.0,25.871,5.900000,AFG,South Asia,L
4,Afghanistan,2012,69.100000,51.434547,0.005029,1873.153946,0.301822,33.0,25.468,5.454545,AFG,South Asia,L


In [9]:
#add a year categorical variable for the pooled OLS analysis
year = pd.Categorical(digital['date'])




In [10]:
year

[2016, 2015, 2014, 2013, 2012, ..., 1999, 1998, 1997, 1996, 1995]
Length: 4136
Categories (22, int64): [1995, 1996, 1997, 1998, ..., 2013, 2014, 2015, 2016]

In [11]:
digital.set_index(['country', 'date'], inplace = True)
digital['year'] = year


In [12]:
digital.head()

Access Electricity  Cellular %  Fixed broadband %  \
country     date                                                      
Afghanistan 2016                 NaN   66.003744           0.026890   
            2015                 NaN   61.577682           0.022080   
            2014           89.500000   58.845471           0.004795   
            2013           75.154373   55.012226           0.004910   
            2012           69.100000   51.434547           0.005029   

                  GDP pcp PPP  Landline %  Secure Servers  Urbanisation  \
country     date                                                          
Afghanistan 2016  1876.544682    0.348892            49.0        27.132   
            2015  1861.124332    0.343677            46.0        26.703   
            2014  1875.447407    0.325861            32.0        26.282   
            2013  1877.411953    0.313466            30.0        25.871   
            2012  1873.153946    0.301822            33.0        25.468   

                  internet_users Code      Region cgroup  year  
country     date                                                
Afghanistan 2016       10.595726  AFG  South Asia      L  2016  
            2015        8.260000  AFG  South Asia      L  2015  
            2014        7.000000  AFG  South Asia      L  2014  
            2013        5.900000  AFG  South Asia      L  2013  
            2012        5.454545  AFG  South Asia      L  2012

In [13]:
#for analysis: need to add an intercept
digital['intercept'] = 1
#alternative: sm.add_constant

In [14]:
digital.head()

Access Electricity  Cellular %  Fixed broadband %  \
country     date                                                      
Afghanistan 2016                 NaN   66.003744           0.026890   
            2015                 NaN   61.577682           0.022080   
            2014           89.500000   58.845471           0.004795   
            2013           75.154373   55.012226           0.004910   
            2012           69.100000   51.434547           0.005029   

                  GDP pcp PPP  Landline %  Secure Servers  Urbanisation  \
country     date                                                          
Afghanistan 2016  1876.544682    0.348892            49.0        27.132   
            2015  1861.124332    0.343677            46.0        26.703   
            2014  1875.447407    0.325861            32.0        26.282   
            2013  1877.411953    0.313466            30.0        25.871   
            2012  1873.153946    0.301822            33.0        25.468   

                  internet_users Code      Region cgroup  year  intercept  
country     date                                                           
Afghanistan 2016       10.595726  AFG  South Asia      L  2016          1  
            2015        8.260000  AFG  South Asia      L  2015          1  
            2014        7.000000  AFG  South Asia      L  2014          1  
            2013        5.900000  AFG  South Asia      L  2013          1  
            2012        5.454545  AFG  South Asia      L  2012          1

In [15]:
exog = digital.drop(['internet_users', 'Code'], axis = 1)
###need to convert Region and cgroup to factor


In [16]:
exog['factor_cgroup'] = exog['cgroup'].astype('category')
exog['factor_region'] = exog['Region'].astype('category')
dummies = pd.get_dummies(exog[['factor_cgroup', 'factor_region']], drop_first=True)
exog = pd.concat([exog, dummies], axis = 1)
exog.head()

Access Electricity  Cellular %  Fixed broadband %  \
country     date                                                      
Afghanistan 2016                 NaN   66.003744           0.026890   
            2015                 NaN   61.577682           0.022080   
            2014           89.500000   58.845471           0.004795   
            2013           75.154373   55.012226           0.004910   
            2012           69.100000   51.434547           0.005029   

                  GDP pcp PPP  Landline %  Secure Servers  Urbanisation  \
country     date                                                          
Afghanistan 2016  1876.544682    0.348892            49.0        27.132   
            2015  1861.124332    0.343677            46.0        26.703   
            2014  1875.447407    0.325861            32.0        26.282   
            2013  1877.411953    0.313466            30.0        25.871   
            2012  1873.153946    0.301822            33.0        25.468   

                      Region cgroup  year                ...                 \
country     date                                         ...                  
Afghanistan 2016  South Asia      L  2016                ...                  
            2015  South Asia      L  2015                ...                  
            2014  South Asia      L  2014                ...                  
            2013  South Asia      L  2013                ...                  
            2012  South Asia      L  2012                ...                  

                  factor_region factor_cgroup_L factor_cgroup_LM  \
country     date                                                   
Afghanistan 2016     South Asia               1                0   
            2015     South Asia               1                0   
            2014     South Asia               1                0   
            2013     South Asia               1                0   
            2012     South Asia               1                0   

                  factor_cgroup_UM  factor_region_Europe & Central Asia  \
country     date                                                          
Afghanistan 2016                 0                                    0   
            2015                 0                                    0   
            2014                 0                                    0   
            2013                 0                                    0   
            2012                 0                                    0   

                  factor_region_Latin America & Caribbean  \
country     date                                            
Afghanistan 2016                                        0   
            2015                                        0   
            2014                                        0   
            2013                                        0   
            2012                                        0   

                  factor_region_Middle East & North Africa  \
country     date                                             
Afghanistan 2016                                         0   
            2015                                         0   
            2014                                         0   
            2013                                         0   
            2012                                         0   

                  factor_region_North America  factor_region_South Asia  \
country     date                                                          
Afghanistan 2016                            0                         1   
            2015                            0                         1   
            2014                            0                         1   
            2013                            0                         1   
            2012                            0                         1   

                  factor_region_Sub-Saharan Afri

In [17]:
exog.columns

Index(['Access Electricity', 'Cellular %', 'Fixed broadband %', 'GDP pcp PPP',
       'Landline %', 'Secure Servers', 'Urbanisation', 'Region', 'cgroup',
       'year', 'intercept', 'factor_cgroup', 'factor_region',
       'factor_cgroup_L', 'factor_cgroup_LM', 'factor_cgroup_UM',
       'factor_region_Europe & Central Asia',
       'factor_region_Latin America & Caribbean',
       'factor_region_Middle East & North Africa',
       'factor_region_North America', 'factor_region_South Asia',
       'factor_region_Sub-Saharan Africa'],
      dtype='object')

In [18]:
exog.drop(['Region', 'cgroup', 'year', 'factor_cgroup', 'factor_region'], axis = 1, inplace = True)

In [19]:
exog.head()
#including the year variable makes matrix not have full column rank, as year is already in the index (date)


Access Electricity  Cellular %  Fixed broadband %  \
country     date                                                      
Afghanistan 2016                 NaN   66.003744           0.026890   
            2015                 NaN   61.577682           0.022080   
            2014           89.500000   58.845471           0.004795   
            2013           75.154373   55.012226           0.004910   
            2012           69.100000   51.434547           0.005029   

                  GDP pcp PPP  Landline %  Secure Servers  Urbanisation  \
country     date                                                          
Afghanistan 2016  1876.544682    0.348892            49.0        27.132   
            2015  1861.124332    0.343677            46.0        26.703   
            2014  1875.447407    0.325861            32.0        26.282   
            2013  1877.411953    0.313466            30.0        25.871   
            2012  1873.153946    0.301822            33.0        25.468   

                  intercept  factor_cgroup_L  factor_cgroup_LM  \
country     date                                                 
Afghanistan 2016          1                1                 0   
            2015          1                1                 0   
            2014          1                1                 0   
            2013          1                1                 0   
            2012          1                1                 0   

                  factor_cgroup_UM  factor_region_Europe & Central Asia  \
country     date                                                          
Afghanistan 2016                 0                                    0   
            2015                 0                                    0   
            2014                 0                                    0   
            2013                 0                                    0   
            2012                 0                                    0   

                  factor_region_Latin America & Caribbean  \
country     date                                            
Afghanistan 2016                                        0   
            2015                                        0   
            2014                                        0   
            2013                                        0   
            2012                                        0   

                  factor_region_Middle East & North Africa  \
country     date                                             
Afghanistan 2016                                         0   
            2015                                         0   
            2014                                         0   
            2013                                         0   
            2012                                         0   

                  factor_region_North America  factor_region_South Asia  \
country     date                                                          
Afghanistan 2016                            0                         1   
            2015                            0                         1   
            2014                            0                         1   
            2013                            0                         1   
            2012                            0                         1   

                  factor_region_Sub-Saharan Africa  
country     date                                    
Afghanistan 2016                                 0  
            2015                                 0  
            2014                                 0  
            2013                                 0  
            2012                                 0

For comparison purpose, I will estimate the following models: 
1. Pooled OLS as baseline
2. Random Effects 
3. The between estimator
4. Fixed effect (time or entity or both)
5. First difference
Then, I will compare all these diference models

In [20]:
#1 Pooled OLS
from linearmodels.panel import PooledOLS
mod = PooledOLS(digital['internet_users'], exog)
pooled_res = mod.fit()
pooled_res


/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/linearmodels/utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,internet_users,R-squared:,0.8785
Estimator:,PooledOLS,R-squared (Between):,0.9085
No. Observations:,1903,R-squared (Within):,0.7534
Date:,"Sun, Apr 22 2018",R-squared (Overall):,0.8785
Time:,22:52:50,Log-likelihood,-6969.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,852.42
Entities:,185,P-value,0.0000
Avg Obs:,10.286,Distribution:,"F(16,1886)"
Min Obs:,1.0000,,
Max Obs:,13.000,F-statistic (robust):,852.42


In [21]:
#2 Random Effects
##Q: Does it make sense to include the year variable here? 
from linearmodels.panel import RandomEffects
mod = RandomEffects(digital['internet_users'], exog)
re_res = mod.fit()

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/linearmodels/utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [22]:
re_res

Dep. Variable:,internet_users,R-squared:,0.8008
Estimator:,RandomEffects,R-squared (Between):,0.8804
No. Observations:,1903,R-squared (Within):,0.7860
Date:,"Sun, Apr 22 2018",R-squared (Overall):,0.8643
Time:,22:52:50,Log-likelihood,-6212.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,473.73
Entities:,185,P-value,0.0000
Avg Obs:,10.286,Distribution:,"F(16,1886)"
Min Obs:,1.0000,,
Max Obs:,13.000,F-statistic (robust):,490.50


In [23]:
re_res.variance_decomposition

Effects                   42.743974
Residual                  38.315311
Percent due to Effects     0.527317
Name: Variance Decomposition, dtype: float64

In [24]:
re_res.theta.head()

,theta
entity,
Afghanistan,0.610098
Albania,0.699039
Algeria,0.736358
Angola,0.699039
Antigua and Barbuda,0.699039


In [26]:
#3 Between Estimator
from linearmodels.panel import BetweenOLS
mod = BetweenOLS(digital['internet_users'], exog)
be_res = mod.fit()

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/linearmodels/utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [27]:
be_res

Dep. Variable:,internet_users,R-squared:,0.9222
Estimator:,BetweenOLS,R-squared (Between):,0.9222
No. Observations:,185,R-squared (Within):,0.6265
Date:,"Sun, Apr 22 2018",R-squared (Overall):,0.8579
Time:,22:53:50,Log-likelihood,-611.64
Cov. Estimator:,Unadjusted,,
,,F-statistic:,124.42
Entities:,185,P-value,0.0000
Avg Obs:,10.286,Distribution:,"F(16,168)"
Min Obs:,1.0000,,
Max Obs:,13.000,F-statistic (robust):,124.42


In [ ]:
#4 Panel with time effects
from linearmodels.panel import PanelOLS
mod = PanelOLS(digital['internet_users'], test_exog, time_effects = True)
fet_res = mod.fit()

In [ ]:
fet_res

Panel with entity effects does not run, because one variable is fully accounted for. Most likely culprit: 
Region

In [ ]:
fe_exog = test_exog.drop(["Region", 'intercept'], axis = 1)

In [ ]:
#4. Fixed effect without region
mod = PanelOLS(digital['internet_users'], fe_exog, entity_effects = True)
fe_res = mod.fit(cov_type ='robust')

In [ ]:
fe_res

In a later analysis: look at the different options: https://bashtage.github.io/linearmodels/doc/panel/models.html#linearmodels.panel.model.PanelOLS

In [ ]:
#4. Fixed effect time and entity(might need to exclude cgroup) 
mod = PanelOLS(digital['internet_users'], fe_exog, entity_effects = True, time_effects = True)
fetm_res = mod.fit(cov_type ='robust')

In [ ]:
fetm_res

In [ ]:
#5. First difference
from linearmodels.panel import FirstDifferenceOLS
#need to exclude time_invariant variables, in particular region 
mod = FirstDifferenceOLS(digital['internet_users'], fd_exog)
fd_res = mod.fit()

In [ ]:
fd_res

In [ ]:
from linearmodels.panel import compare
compare({'BE':be_res,'RE':re_res,'FE': fe_res, 'Pooled':pooled_res})

Is random or fixed effects more appropriate?

In [ ]:
#quick and dirty fix: 
import statsmodels.formula.api as sm
from scipy import stats
def hausman(fe, reparams, recov):
    """
    Compute hausman test for fixed effects/random effects models

    b = beta_fe
    B = beta_re

    From theory we have that b is always consistent, but B is consistent
    under the alternative hypothesis and efficient under the null.

    The test statistic is computed as

    z = (b - B)' [V_b - v_B^{-1}](b - B)

    The statistic is distributed z \sim \chi^2(k), where k is the number
 import statsmodels.formula.api as sm
from scipy import stats   of regressors in the model.

    Parameters
    ==========
    fe : statsmodels.regression.linear_panel.PanelLMWithinResults
        The results obtained by using sm.PanelLM with the
        method='within' option.

    re : statsmodels.regression.linear_panel.PanelLMRandomResults
        The results obtained by using sm.PanelLM with the
        method='swar' option.

    Returns
    =======
    chi2 : float
        The test statistic

    df : int
        The number of degrees of freedom for the distribution of the
        test statistic

    pval : float
        The p-value associated with the null hypothesis

    Notes
    =====
    The null hypothesis supports the claim that the random effects
    estimator is "better". If we reject this hypothesis it is the same
    as saying we should be using fixed effects because there are
    systematic differences in the coefficients.

    """

    # Pull data out
    b = fe.params
    B = reparams
    v_b = fe.cov
    v_B = recov

    # NOTE: find df. fe should toss time-invariant variables, but it
    #       doesn't. It does return garbage so we use that to filter
    df = b[np.abs(b) < 1e8].size

    # compute test statistic and associated p-value
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B))
    test = la.inv(v_b - v_B).dot(b - B)
    pval = stats.chi2.sf(chi2, df)

    return chi2, df, pval

#Code adapted from Econtools


In [ ]:
re_res_params = re_res.params.drop(re_res.params.index.difference(fe_res.params.index))

In [ ]:
re_res_cov = re_res.cov.drop(re_res.cov.index.difference(fe_res.cov.index), axis = 0)
re_res_cov.drop(re_res.cov.index.difference(fe_res.cov.index), axis = 1, inplace = True)

In [ ]:
hausman(fe_res, re_res_params, re_res_cov)

The test statistic is large, and we reject the null at a reasonable confidence level, therefore random effects is preferred. The most likely explanasion is our inclusion of region dummies and high/low income, which are likelyto absorb most variation from the entities.

## Integrating with R

In [6]:
%load_ext rpy2.ipython
pandas2ri.activate()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [7]:
r_dataframe = pandas2ri.py2ri(digital)

In [9]:
%R
install.packages("plm")
library(plm)

/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in dev.off() : cannot shut down device 1 (the null device)

  warnings.warn(x, RRuntimeWarning)
/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In (function (filename = "Rplot%03d.png", width = 480, height = 480,  :
  warnings.warn(x, RRuntimeWarning)
/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
 
  warnings.warn(x, RRuntimeWarning)
/home/toni/.conda/envs/digital_divide/lib/python3.6/site-packages/r

RRuntimeError: Error in dev.off() : cannot shut down device 1 (the null device)
